In [12]:
import pandas as pd
import os
import yfinance as yf
from sec_cik_mapper import StockMapper

The JSON files come in two main partitions (need to find out exactly what they mean)

- DEI

- US-GAAP

General Structure of the JSON file:

- [Partition]
    - Heading Name
        - Label (more verbose heading)
        - Description
        - Units
            - Unit of Measurement
                - ...
                - Val
                - Form
                - Filed (date)
                - ...
    

In [16]:
minYear = 2023

columnNames = ['CIK', 'heading', 'units', 'value', 'frame']
# other columns if needed: fy, qtr

## Read Company Financials JSON file

This function takes in a DataFrame (which should just be what is returned by called pd.read_json() on a company's JSON file) and returns another DataFrame with the JSON data transformed into a tabular format

In [17]:

def read_company_financials_json(df: pd.DataFrame):
    # Pull out the company's name and CIK
    entityName = df.iloc[0,1]
    entityCIK = str(df.iloc[0,0])
    
    # Initialise a dataframe to hold the data
    companyDF = pd.DataFrame(columns=columnNames)
    quarters = {'03': 'Q1', '06': 'Q2', '09': 'Q3', '12': 'Q4'}

    for i in range(len(df)):
        partitionName = df.index[i]
        partitionName = partitionName.upper()
        partition = df.iloc[i,2]
        subGroups = partition.items()

        intermediateDF = pd.DataFrame(columns=columnNames)

        if partitionName == 'US-GAAP':
            # Each of DEI, invest, US-GAAP is broken down into subgroups
            for group in subGroups:
                # Each subgroup has a heading, detailed description, and unit of measurement
                row = {}
                heading = group[0]
                # row['company'] = entityName
                row['CIK'] = entityCIK
                row['heading'] = heading
                desc = group[1]['description']

                # The units component contains the actual measurement and the associated metadata (in the 'records' list)
                units = group[1]['units']
                for unit, records in units.items():
                    row['units'] = unit

                    # We have a record for every filing date
                    for record in records:
                        periodEndYear, periodEndMonth, _ = record['end'].split('-')
                        periodStartYear, periodStartMonth = (0, 0)
                        try:
                            periodStartYear, periodStartMonth, _ = record['start'].split('-')
                        except KeyError: 
                            pass
                        
                        if record['form'] == '10-Q' and int(periodEndYear) >= 2018: 
                            if 'frame' in record.keys():
                                row['value'] = round(float(record['val']), 2)
                                row['frame'] = record['frame']

                                # Append the row to the main dataframe
                                nextIdx = len(intermediateDF)
                                intermediateDF.loc[nextIdx] = row
                            elif (periodEndMonth in quarters.keys() and 
                                  ((int(periodEndMonth) - int(periodStartMonth)) == 3) and
                                  (int(periodEndYear) == periodStartYear)):
                                row['value'] = round(float(record['val']), 2)
                                row['frame'] = "CY" + periodEndYear + quarters[periodEndMonth]

                                # Append the row to the main dataframe
                                nextIdx = len(intermediateDF)
                                intermediateDF.loc[nextIdx] = row                                
                            
        companyDF = pd.concat([companyDF, intermediateDF], ignore_index=True)
                        
    return companyDF


## Read through each company's JSON file and append to masterDF

In [18]:
#folderPath = '.'
folderPath = 'companyfacts/Biggest Files'

masterDF = pd.DataFrame(columns=columnNames)
dimCompany = pd.DataFrame(columns=['company', 'CIK'])

counter = 0
for filename in os.listdir(folderPath):
    counter += 1
    if filename.endswith('.json') and counter < 100:
        filePath = os.path.join(folderPath, filename)
        df = pd.read_json(filePath)
        
        entityName = df.iloc[0,1]
        entityCIK = str(df.iloc[0,0])
        companyIdx = len(dimCompany)
        dimCompany.loc[companyIdx] = {'company': entityName, 'CIK': entityCIK}
        
        companyDF = read_company_financials_json(df)
        
        masterDF = pd.concat([masterDF, companyDF], ignore_index=True)
        
        
masterDF
masterDF.to_csv('testDF.csv', index=False)
#dimCompany.to_csv('dimCompany.csv', index=False)

## CIK to Ticker mapping

We have all of the company's CIKs from the SEC EDGAR data. This can be mapped to their stock ticker with the sec_cik_mapper package. This ticker can then be mapped to the company's sector and industry with the yfinance API

In [43]:
# Dimensional table for tickers that can be joined to stock price data
dimTickers = pd.DataFrame(columns=["ticker", "CIK"])

# Dimensional table for mapping companies to their sectors
dimSectors = pd.DataFrame(columns=["sector", "industry", "CIK"])

# Create the CIK to ticker mapping from the sec_cik_mapper package
mapper = StockMapper()
cikToTickers = mapper.cik_to_tickers

tickerIdx = 0
sectorIdx = 0

# Iterate through all companies
for i in range(len(dimCompany)):
    cik = dimCompany.iloc[i,1]
    companyName = dimCompany.iloc[i,0]
    
    # Find the corresponding ticker for the company's CIK
    for key in cikToTickers.keys():
        if cik in key:
            for ticker in cikToTickers[key]:
                if "-" not in ticker:
                    # Add the CIK and ticker to dimTickers
                    dimTickers.loc[tickerIdx] = {"ticker": ticker, "CIK": cik}
                    tickerIdx += 1
                    
                    # Use yfinance API to get the company's sector/industry with their ticker
                    stockInfo = yf.Ticker(ticker).get_info()
                    try:
                        stockSector = stockInfo["sector"]
                    except KeyError:
                        stockSector = None
                    
                    try:
                        stockIndustry = stockInfo["industry"]
                    except KeyError:
                        stockIndustry = None
                    
                    dimSectors.loc[sectorIdx] = {"sector": stockSector, 
                                                 "industry": stockIndustry,
                                                 "CIK": cik}
                    sectorIdx += 1
                             

In [46]:
dimTickers.to_csv('dimTickers.csv', index=False)
dimSectors.to_csv('dimSectors.csv', index=False)